In [92]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [93]:
import pandas as pd
import s3fs
df = pd.read_csv('s3://proyectocanopy/alarma_transparencia.csv')

In [5]:
df.shape

(22725, 18)

# Transparencia

In [44]:
df_transparency = pd.read_csv('s3://proyectocanopy/alarma_transparencia.csv')
df_transparency.head()

,Unnamed: 0,Unnamed: 0.1,ids,fechas,regiones,departamento,municipio,entidades,contratista_ids,contratista_name,valores,detalles,reporta_parties,reporta_planning,uri,plataforma,reporta_items,departamento_def
0,0,98,20-12-10962842,2020-07-28T15:54:33.000Z,ARAUCA,Arauca,ARAUCA,ARAUCA - UNIDAD ADMINISTRATIVA ESPECIAL DE SAL...,111677609,RUIZ BARCO SOFIA SAHARAY,7189350.0,PRESTACION DE SERVICIOS AUXILIARES COMO APOYO...,SI_REPORTA,SI_REPORTA,https://www.contratos.gov.co/consultas/detalle...,SECOPI,SI_REPORTA,arauca
1,1,140,20-12-10977749,2020-08-03T06:43:56.000Z,NARIÑO,Nariño,BARBACOAS,NARIÑO - BARBACOAS - ALCALDIA,94534533,SILVIO RUA QUIÑONES,1200000.0,BRINDAR APOYO COMO AUXILIAR OPERATIVO DE LA AL...,SI_REPORTA,SI_REPORTA,https://www.contratos.gov.co/consultas/detalle...,SECOPI,SI_REPORTA,nariño
2,2,247,20-4-10903430,2020-07-07T17:21:03.000Z,SANTANDER,NaN,NaN,SANTANDER - COLEGIO INTEGRADO NUESTRA SEÑORA D...,900579939,SOLUCIONES & TECNOLOGIA SAS,9486000.0,"INSUMOS DE FOTOCOPIADORA, Y SUMINISTRO INTEGRA...",SI_REPORTA,SI_REPORTA,https://www.contratos.gov.co/consultas/detalle...,SECOPI,SI_REPORTA,NaN
3,3,326,20-12-10903397,2020-07-07T17:13:18.000Z,BOLIVAR,Bolívar,SAN MARTÍN DE LOBA,BOLIVAR - SAN MARTIN DE LOBA - ALCALDIA,23106451,MAVELIS BANDERA BELAIDES,7500000.0,Prestación de servicios de Auxiliar de Enferme...,SI_REPORTA,SI_REPORTA,https://www.contratos.gov.co/consultas/detalle...,SECOPI,SI_REPORTA,bolívar
4,4,339,20-11-10875699,2020-06-30T10:38:07.000Z,BOGOTÁ D.C.,Bogotá D.C.,Bogotá D.C.,BOGOTÁ D.C. - IED. COLEGIO GABRIEL BETANCOURT ...,900682841,EDUCACTIVA SAS,59937000.0,COMPRA DE CARTILLAS GRADOS DE JARDIN A QUI...,SI_REPORTA,SI_REPORTA,https://www.contratos.gov.co/consultas/detalle...,SECOPI,SI_REPORTA,bogotá


In [45]:
# Creating alarm variables
df_transparency['has_alarm'] = (df_transparency['reporta_items']!='SI_REPORTA')
# Filtering
df_transparency = df_transparency[df_transparency['has_alarm']]

# Formatting with $ 
df_transparency['valores'] = pd.to_numeric(df_transparency['valores'], errors='coerce')
df_transparency['valores'] = df_transparency['valores'].map('${:,.0f}'.format)
# Formatting descripcion
df_transparency['detalles'] = df_transparency['detalles'].str.capitalize()
# Formatting fechas with only date
df_transparency['date'] = pd.to_datetime(df_transparency['fechas']).dt.date
# Getting dates' month
df_transparency['month'] =pd.to_datetime(df_transparency['fechas']).dt.to_period("M")
# Capitalizing names of columns
df_transparency['municipio'] = df_transparency['municipio'].str.title()
df_transparency['departamento'] = df_transparency['departamento'].str.title()
df_transparency['contratista_name'] = df_transparency['contratista_name'].str.title()

df_transparency = df_transparency.rename(columns={
    'entidades': 'Nombre de la entidad',
    'departamento': 'Departamento',
    'municipio': 'Municipio',
    'date': 'Fecha del contrato',
    'detalles': 'Descripcion del contrato',
    'contratista_name': 'Proveedor adjudicado',
    'contratista_ids': 'ID Proveedor',
    'uri': 'SECOP URL',
    'valores': 'Valor del contrato',
    'month': 'Mes del contrato'
})

df_transparency_final = df_transparency[[ 'Fecha del contrato', 'Departamento', 'Municipio', 
                                   'Nombre de la entidad', 'Proveedor adjudicado', 
                                    'ID Proveedor', 'Valor del contrato',
                                     'Descripcion del contrato',  'SECOP URL']]
df_transparency_final = df_transparency_final.sort_values(by='Fecha del contrato', ascending=False)

/Users/lmontesmarti/Documents/GitHub/canopy-demo/venv/lib/python3.7/site-packages/pandas/core/arrays/datetimes.py:1091: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  UserWarning,


In [46]:
df_transparency_final.head()

,Fecha del contrato,Departamento,Municipio,Nombre de la entidad,Proveedor adjudicado,ID Proveedor,Valor del contrato,Descripcion del contrato,SECOP URL
18725,2020-07-05,Tolima,Ibagué,TOLIMA - UNIVERSIDAD DEL TOLIMA,Hamilton Julian Barbosa Vinasco,111052812,"$8,000,000",Apoyo a la gestión para procesar las diferente...,https://www.contratos.gov.co/consultas/detalle...
3351,2020-06-18,NaN,NaN,RISARALDA - INSTITUCION EDUCATIVA RODRIGO AREN...,Luisa Fernanda Marin Zuluaga,100471954,"$11,260,000","Suministrar, dos computadores todo en uno leno...",https://www.contratos.gov.co/consultas/detalle...
16740,2020-05-26,Nariño,Ipiales,NARIÑO - IPIALES - ALCALDIA,Alba Susana Timaran Tibata,108590121,"$11,811,282",Prestación de servicios profesionales de enfer...,NaN
16755,2020-05-26,Antioquia,Girardota,ANTIOQUIA - GIRARDOTA - ALCALDIA,Luis Fernando Castaño Orozco,71646152,"$12,000,000",Prestación de servicios profesionales y de a`p...,NaN
3546,2020-05-26,Bolívar,Arjona,BOLIVAR - ESE HOSPITAL LOCAL ARJONA,Alvaro Alfonso Arrieta Perez,9097905,"$11,638,413",Adición del contrato de obra para la adecuaci...,NaN


In [48]:
print(df_transparency_final['SECOP URL'].isna().sum())
print(len(df_transparency_final))

10259
10261


## Graph for transparency

In [50]:
graph_df = df_transparency.groupby(['Mes del contrato']).size().reset_index(name='Num contratos')
# graph_df = graph_df.pivot_table(index='Mes del contrato', columns='Razón alarma', values='Num contracts')
graph_df

,Mes del contrato,Num contratos
0,2020-01,45
1,2020-02,69
2,2020-03,1298
3,2020-04,5708
4,2020-05,3139
5,2020-06,1
6,2020-07,1


In [52]:
fig = go.Figure(data=[
    go.Bar(x=graph_df['Mes del contrato'].astype(str), y=graph_df['Num contratos']),
])
# Change the bar mode
fig.update_layout(
    xaxis_tickformat = '%B <br>%Y'
)

fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = graph_df['Mes del contrato'].astype(str).unique(),
        ticktext = graph_df['Mes del contrato'].astype(str).unique()
    )
)
fig.update_layout(
    font=dict(
        color="#252525",
        family="Roboto"
    ),
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    xaxis_title="Fecha del contrato",
    margin={"r":0,"t":0,"l":0,"b":0}
)
fig.show()

# 2. Financiación de campañas

In [53]:
financiacion = pd.read_csv('s3://proyectocanopy/alarma_financiacion.csv')
financiacion.head()

,Unnamed: 0,Unnamed: 0.1,ids,fechas,municipio_contrato,entidades,contratista_ids,contratista_name,valores,detalles,...,circunscripcion.electoral,organizacion.politica,identificacion.candidato,nombre.candidato,elegido,valor,tipo.donacion,concepto.del.ingreso,identificacion_aportante,aportante.original.nombre
0,0,1457,20-4-10874698,2020-06-29T16:44:19.000Z,NaN,META - INSTITUCION EDUCATIVA MANUELA BELTRAN -...,890900943,COLOMBIANA DE COMERCIO SA,5415800.0,"SUMINISTRO DE COMPUTADORES PORTATILES, ANTIVIR...",...,Municipal,PARTIDO ALIANZA VERDE - PARTIDO ALIANZA SOCIAL...,71587929,FABIO ALONSO SALAZAR JARAMILLO,No,"$20,000,000",Donación,DONACION PERSONA JURIDICA,890900943,COLOMBIANA DE COMERCIO SA
1,1,1457,20-4-10874698,2020-06-29T16:44:19.000Z,NaN,META - INSTITUCION EDUCATIVA MANUELA BELTRAN -...,890900943,COLOMBIANA DE COMERCIO SA,5415800.0,"SUMINISTRO DE COMPUTADORES PORTATILES, ANTIVIR...",...,Municipal,PARTIDO CENTRO DEMOCRATICO - PARTIDO CONSERVAD...,15440458,ANDRES JULIAN RENDON CARDONA,Si,"$5,000,000",Donación,DONACION PARTICULAR,890900943,COLOMBIANA DE COMERCIO S.A
2,2,1457,20-4-10874698,2020-06-29T16:44:19.000Z,NaN,META - INSTITUCION EDUCATIVA MANUELA BELTRAN -...,890900943,COLOMBIANA DE COMERCIO SA,5415800.0,"SUMINISTRO DE COMPUTADORES PORTATILES, ANTIVIR...",...,Municipal,CENTRO DEMOCRATICO,19463238,FRANCISCO SANTOS CALDERON,No,"$25,000,000",Donación,DONACIÓN,890900943,COLOMBIANA DE COMERCIO S.A.
3,3,1457,20-4-10874698,2020-06-29T16:44:19.000Z,NaN,META - INSTITUCION EDUCATIVA MANUELA BELTRAN -...,890900943,COLOMBIANA DE COMERCIO SA,5415800.0,"SUMINISTRO DE COMPUTADORES PORTATILES, ANTIVIR...",...,Municipal,RECUPEREMOS BOGOTA,19333686,ENRIQUE PEÑALOSA LONDOÑO,Si,"$25,000,000",Donación,DONACIÓN,890900943,COLOMBIANA DE COMERCIO S.A
4,4,1457,20-4-10874698,2020-06-29T16:44:19.000Z,NaN,META - INSTITUCION EDUCATIVA MANUELA BELTRAN -...,890900943,COLOMBIANA DE COMERCIO SA,5415800.0,"SUMINISTRO DE COMPUTADORES PORTATILES, ANTIVIR...",...,Municipal,CREEMOS CALI,14446558,NORMAN MAURICE ARMITAGE CADAVID,Si,"$10,000,000",Donación,DONACION CHQ 174416 BANCOLOMBIA,890900943,COLOMBIANA DE COMERCIO S.A.


In [54]:
financiacion[['corporacion.o.cargo', 'circunscripcion.electoral',
       'organizacion.politica', 'identificacion.candidato', 'nombre.candidato',
       'elegido', 'valor', 'tipo.donacion', 'concepto.del.ingreso',
       'identificacion_aportante', 'aportante.original.nombre']]

,corporacion.o.cargo,circunscripcion.electoral,organizacion.politica,identificacion.candidato,nombre.candidato,elegido,valor,tipo.donacion,concepto.del.ingreso,identificacion_aportante,aportante.original.nombre
0,Alcaldía,Municipal,PARTIDO ALIANZA VERDE - PARTIDO ALIANZA SOCIAL...,71587929,FABIO ALONSO SALAZAR JARAMILLO,No,"$20,000,000",Donación,DONACION PERSONA JURIDICA,890900943,COLOMBIANA DE COMERCIO SA
1,Alcaldía,Municipal,PARTIDO CENTRO DEMOCRATICO - PARTIDO CONSERVAD...,15440458,ANDRES JULIAN RENDON CARDONA,Si,"$5,000,000",Donación,DONACION PARTICULAR,890900943,COLOMBIANA DE COMERCIO S.A
2,Alcaldía,Municipal,CENTRO DEMOCRATICO,19463238,FRANCISCO SANTOS CALDERON,No,"$25,000,000",Donación,DONACIÓN,890900943,COLOMBIANA DE COMERCIO S.A.
3,Alcaldía,Municipal,RECUPEREMOS BOGOTA,19333686,ENRIQUE PEÑALOSA LONDOÑO,Si,"$25,000,000",Donación,DONACIÓN,890900943,COLOMBIANA DE COMERCIO S.A
4,Alcaldía,Municipal,CREEMOS CALI,14446558,NORMAN MAURICE ARMITAGE CADAVID,Si,"$10,000,000",Donación,DONACION CHQ 174416 BANCOLOMBIA,890900943,COLOMBIANA DE COMERCIO S.A.
...,...,...,...,...,...,...,...,...,...,...,...
917,Alcaldía,Municipal,PARTIDO CONSERVADOR COLOMBIANO,8106620,HUGO ANDRES DIAZ VILLA,No,"$5,000,000",Donación,DONACION A LA CAMPAÑA PARA LA ALCALDIA DEL MUN...,800093761,TAX INDIVIDUAL S.A
918,Concejo,Municipal,PARTIDO LIBERAL COLOMBIANO,71310233,ARIES WEST ROWE MATEUS,Si,"$3,000,000",Donación,DONACION PARA GASTOS DE LA CAMPAÑA AL CONCEJO,800093761,EMPRESA TRANSPORTADORA DE TAXIS INDIVIDUAL SA
919,Senado de la República,Nacional Ordinaria,MOVIMIENTO POLITICO TODOS SOMOS COLOMBIA - TCS,71451657,GUILLERMO ENRIQUE GONZALEZ OCHOA,No,"$500,000",Donación,Donación como apoyo a la campaña del candidato...,900270192,CORPORACIÓN VISIÓN GENTE
920,Concejo,Municipal,PARTIDO SOCIAL DE UNIDAD NACIONAL PARTIDO DE LA U,42679912,ADRIANA MARIA JIMENEZ GOMEZ,No,"$1,000,000",Aporte,APORTE CAMPAÑA DE SU PROPIO PATRIMONIO,42679912,ADRIANA MARIA JIMENEZ GOMEZ


In [55]:
financiacion.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'ids', 'fechas', 'municipio_contrato',
       'entidades', 'contratista_ids', 'contratista_name', 'valores',
       'detalles', 'uri', 'departamento_def', 'corporacion.o.cargo',
       'circunscripcion.electoral', 'organizacion.politica',
       'identificacion.candidato', 'nombre.candidato', 'elegido', 'valor',
       'tipo.donacion', 'concepto.del.ingreso', 'identificacion_aportante',
       'aportante.original.nombre'],
      dtype='object')

In [56]:
financiacion.shape

(922, 23)

In [60]:
financiacion[financiacion['organizacion.politica'].str.contains('CENTRO')]

,Unnamed: 0,Unnamed: 0.1,ids,fechas,municipio_contrato,entidades,contratista_ids,contratista_name,valores,detalles,...,circunscripcion.electoral,organizacion.politica,identificacion.candidato,nombre.candidato,elegido,valor,tipo.donacion,concepto.del.ingreso,identificacion_aportante,aportante.original.nombre
1,1,1457,20-4-10874698,2020-06-29T16:44:19.000Z,NaN,META - INSTITUCION EDUCATIVA MANUELA BELTRAN -...,890900943,COLOMBIANA DE COMERCIO SA,5415800.0,"SUMINISTRO DE COMPUTADORES PORTATILES, ANTIVIR...",...,Municipal,PARTIDO CENTRO DEMOCRATICO - PARTIDO CONSERVAD...,15440458,ANDRES JULIAN RENDON CARDONA,Si,"$5,000,000",Donación,DONACION PARTICULAR,890900943,COLOMBIANA DE COMERCIO S.A
2,2,1457,20-4-10874698,2020-06-29T16:44:19.000Z,NaN,META - INSTITUCION EDUCATIVA MANUELA BELTRAN -...,890900943,COLOMBIANA DE COMERCIO SA,5415800.0,"SUMINISTRO DE COMPUTADORES PORTATILES, ANTIVIR...",...,Municipal,CENTRO DEMOCRATICO,19463238,FRANCISCO SANTOS CALDERON,No,"$25,000,000",Donación,DONACIÓN,890900943,COLOMBIANA DE COMERCIO S.A.
8,8,31997,20-12-10967566,2020-07-29T17:21:28.000Z,NaN,ATLANTICO - INSTITUCION EDUCATIVA DISTRITAL CO...,890900943,COLOMBIANA DE COMERCIO S.A,13941000.0,"Suministro De Nueve (9) Computadores Marca HP,...",...,Municipal,PARTIDO CENTRO DEMOCRATICO - PARTIDO CONSERVAD...,15440458,ANDRES JULIAN RENDON CARDONA,Si,"$5,000,000",Donación,DONACION PARTICULAR,890900943,COLOMBIANA DE COMERCIO S.A
9,9,31997,20-12-10967566,2020-07-29T17:21:28.000Z,NaN,ATLANTICO - INSTITUCION EDUCATIVA DISTRITAL CO...,890900943,COLOMBIANA DE COMERCIO S.A,13941000.0,"Suministro De Nueve (9) Computadores Marca HP,...",...,Municipal,CENTRO DEMOCRATICO,19463238,FRANCISCO SANTOS CALDERON,No,"$25,000,000",Donación,DONACIÓN,890900943,COLOMBIANA DE COMERCIO S.A.
15,15,53728,20-4-10746585,2020-05-09T10:07:42.000Z,SANTA ROSA DEL SUR,CALDAS - INSTITUCION EDUCATIVA GOMEZ FERNANDEZ...,890900943,Colombiana de Comercio S.A,5119000.0,Suministro de Impresora y Resmas de papel con ...,...,Municipal,PARTIDO CENTRO DEMOCRATICO - PARTIDO CONSERVAD...,15440458,ANDRES JULIAN RENDON CARDONA,Si,"$5,000,000",Donación,DONACION PARTICULAR,890900943,COLOMBIANA DE COMERCIO S.A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899,899,82495,20-4-10871561,2020-06-26T13:36:12.000Z,CALI,VALLE DEL CAUCA - SOCIEDAD DE ACUEDUCTOS Y ALC...,890301753,SI S.A.S,2244935.0,SUMINISTRO DE ELEMENTOS DE BIOSEGURIDAD (TOALL...,...,Departamental y Distrito Capital,PARTIDO CENTRO DEMOCRATICO,94503511,CHRISTIAN MUNIR GARCES ALJURE,Si,"$5,000,000",Contribución,CONTRIBUCION,890301753,ALMACENES SI S.A.
901,901,87665,20-4-10634988,2020-03-30T17:06:31.000Z,NaN,ANTIOQUIA - CORPORACION RUTA N MEDELLIN,890980724,FUNDACIoN BANCOLOMBIA,0.0,"Aunar esfuerzos técnicos, administrativos y fi...",...,Municipal,CENTRO DEMOCRATICO,70563378,JUAN CARLOS ROBERTO VELEZ URIBE,No,"$20,000",Donación,DONACION,890980724,FUNDACION BANCOLOMBIA
902,902,87665,20-4-10634988,2020-03-30T17:06:31.000Z,NaN,ANTIOQUIA - CORPORACION RUTA N MEDELLIN,890980724,FUNDACIoN BANCOLOMBIA,0.0,"Aunar esfuerzos técnicos, administrativos y fi...",...,Municipal,CENTRO DEMOCRATICO,70563378,JUAN CARLOS ROBERTO VELEZ URIBE,No,"$19,980,000",Donación,DONACION,890980724,FUNDACION BANCOLOMBIA
907,907,87665,20-4-10634988,2020-03-30T17:06:31.000Z,NaN,ANTIOQUIA - CORPORACION RUTA N MEDELLIN,890980724,FUNDACIoN BANCOLOMBIA,0.0,"Aunar esfuerzos técnicos, administrativos y fi...",...,Municipal,CENTRO DEMOCRATICO,19463238,FRANCISCO SANTOS CALDERON,No,"$20,000,000",Donación,DONACIÓN,890980724,FUNDACION BANCOLOMBIA


In [85]:
df_financiacion = financiacion.copy()



# Formatting with $ 
df_financiacion['valores'] = df_financiacion['valores'].map('${:,.0f}'.format)
# Formatting with capitalization
df_financiacion['detalles'] = df_financiacion['detalles'].str.capitalize()
df_financiacion['contratista_name'] = df_financiacion['contratista_name'].str.title()
df_financiacion['departamento_def'] = df_financiacion['departamento_def'].str.title()
df_financiacion['municipio_contrato'] = df_financiacion['municipio_contrato'].str.title()
df_financiacion['organizacion.politica'] = df_financiacion['organizacion.politica'].str.title()
df_financiacion['nombre.candidato'] = df_financiacion['nombre.candidato'].str.title()

# Formatting nulls of municipio
df_financiacion['municipio_contrato'] = df_financiacion['municipio_contrato'].fillna('')
# Formatting fechas with only date
df_financiacion['date'] = pd.to_datetime(df_financiacion['fechas']).dt.date
# Getting dates' month
df_financiacion['month'] =pd.to_datetime(df_financiacion['fechas']).dt.to_period("M")
# URL temp
df_financiacion['url'] = "https://www.youtube.com/"


df_financiacion = df_financiacion.rename(columns={
    'entidades': 'Nombre de la entidad',
    'departamento_def': 'Departamento contrato',
    'municipio_contrato': 'Municipio',
    'date': 'Fecha del contrato',
    'detalles': 'Descripcion del contrato',
    'contratista_name': 'Nombre Financiador',
    'contratista_ids': 'ID Financiador',
    'url': 'SECOP URL',
    'valores': 'Valor del contrato',
    'valor': 'Valor del aporte',
    'month': 'Mes del contrato',
    'elegido': 'Candidato elegido',
    'organizacion.politica': 'Organización Política',
    'nombre.candidato': 'Nombre Candidato'
})


df_financiacion_final = df_financiacion[['Fecha del contrato', 'Departamento contrato',
                                   'Nombre de la entidad', 'Nombre Financiador', 
                                    'ID Financiador', 'Valor del contrato',
                                     'Descripcion del contrato', 'Valor del aporte',
                                 'Organización Política','Nombre Candidato', 'SECOP URL' ]]
df_financiacion_final = df_financiacion_final.sort_values(by='Fecha del contrato', ascending=False)
df_financiacion_final.head()

/Users/lmontesmarti/Documents/GitHub/canopy-demo/venv/lib/python3.7/site-packages/pandas/core/arrays/datetimes.py:1091: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



,Fecha del contrato,Departamento contrato,Nombre de la entidad,Nombre Financiador,ID Financiador,Valor del contrato,Descripcion del contrato,Valor del aporte,Organización Política,Nombre Candidato,SECOP URL
747,2020-09-30,Valle Del Cauca,VALLE DEL CAUCA - ESE HOSPITAL NUESTRA SEÑORA ...,Distracom S.A,811009788,"$5,000,000",El contratista se obliga con el contratante a ...,"$5,000,000",Partido Centro Democratico,Juan Fernando Espinal Ramirez,https://www.youtube.com/
749,2020-09-30,Valle Del Cauca,VALLE DEL CAUCA - ESE HOSPITAL NUESTRA SEÑORA ...,Distracom S.A,811009788,"$5,000,000",El contratista se obliga con el contratante a ...,"$50,000,000",Partido Social De Unidad Nacional Partido De La U,German Dario Hoyos Giraldo,https://www.youtube.com/
96,2020-09-30,Bogotá,BOGOTÁ D.C. - ALCALDÍA LOCAL LOS MARTIRES,Caja De Compensacion Familiar - Compensar,860066942,"$773,946,197",Compensar se obliga a prestar los servicios re...,"$222,621",Bogota Adelante,Rafael Pardo Rueda,https://www.youtube.com/
748,2020-09-30,Valle Del Cauca,VALLE DEL CAUCA - ESE HOSPITAL NUESTRA SEÑORA ...,Distracom S.A,811009788,"$5,000,000",El contratista se obliga con el contratante a ...,"$10,000,000",Partido Centro Democratico,Paola Andrea Holguin Moreno,https://www.youtube.com/
113,2020-09-29,NaN,ATLANTICO - INSTITUCION EDUCATIVA DISTRITAL KA...,Yesenia Montero Leybas,32844355,"$7,000,000",Suministro de impresión y entrega de fotocopia...,"$1,000,000",Partido Social De Unidad Nacional Partido De La U,Juan Ospino Acuña,https://www.youtube.com/


In [86]:
graph_f_df = df_financiacion.groupby(['Organización Política']).size().reset_index(name='Num contratos')
graph_f_df = graph_f_df.sort_values(by='Num contratos')
graph_f_df = graph_f_df.iloc[-11:-1,:]

fig = px.bar(graph_f_df, x="Num contratos", y="Organización Política", orientation='h',
             height=500)
fig.update_layout(
    font=dict(
        color="#252525",
        family="Roboto"
    ),
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    xaxis_title="Numero de contratos asignados a financiadores",
    yaxis_title='',
    margin={"r":0,"t":0,"l":0,"b":0}
)
fig.show()

# 3. Concentración contratistas

In [91]:
concentracion = pd.read_csv('s3://proyectocanopy/alarma_concentracion_valor_contratistas.csv')
concentracion

,Unnamed: 0,departamento_def,contratista_ids,val_contratista,val_depto,pct_def
0,2,amazonas,7634220,19000000.0,2.301949e+09,0.825388
1,3,amazonas,15877822,419815500.0,2.301949e+09,18.237393
2,4,amazonas,17625757,25200000.0,2.301949e+09,1.094724
3,5,amazonas,100292219,10112000.0,2.301949e+09,0.439280
4,6,amazonas,112120308,6300000.0,2.301949e+09,0.273681
...,...,...,...,...,...,...
15138,15232,vichada,900283540,11226500.0,1.142489e+09,0.982635
15139,15233,vichada,900351693,10408983.0,1.142489e+09,0.911080
15140,15234,vichada,900484765,15000000.0,1.142489e+09,1.312923
15141,15235,vichada,900487314,72000000.0,1.142489e+09,6.302031
